# Image Processing Pipeline Optimization Notebook
This notebook demonstrates how to set up and optimize an image processing pipeline using a large language model (LLM) feedback loop. Each section includes explanations for a university-level understanding.

## 1. Environment Setup and Imports
We import necessary libraries and configure the environment to ensure reproducibility.

In [ ]:
# Imports for environment management and scientific computing
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import pytesseract  # OCR tool

# Print versions for reproducibility
print(f"Python version: {sys.version}")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.__version__}")

## 2. Project Root Path Configuration
We add the project root to `sys.path` for module imports from the repository.

In [ ]:
# Adjust project root to import custom modules
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print(f"Project root added: {project_root}")

## 3. LLM Pipeline Imports
We import the optimized pipeline builders and processing functions from our `lib` directory.

In [ ]:
from lib.image.llm_pipeline import optimize_pipeline, build_pipeline, process_image
from ollama import chat  # Ollama client for LLM interactions

## 4. Defining LLM Models and Parameter Space
We specify which LLMs to test and define the hyperparameter search space for each image processing step.

In [ ]:
# LLM models available for feedback
models = [
    "llama3.2-vision:11b",
    "deepseek-r1:8b",
    "deepseek-r1:32b"
]

# Hyperparameter search ranges for each step
param_space = {
    "denoise": {"h": [5, 10, 15], "template_window": [3, 7]},
    "contrast": {"alpha": [1.0, 1.2, 1.5], "beta": [0, 10, 20]},
    "resize": {"width": [800, 1024, 1280]},
    "normalize": {"clipLimit": [1.0, 2.0], "tileGrid": [(8, 8)]},
    "crop": {"margin": [0, 5, 10]},
    "hash": {}  # Placeholder for future steps
}
print(param_space)

## 5. Generating Random Configuration
Function to sample a random valid configuration from the parameter space.

In [ ]:
def generate_random_config():
    return {
        "use": {
            "rotate": random.choice([True, False]),
            "denoise": True,
            "contrast": random.choice([True, False]),
            "resize": True,
            "normalize": True,
            "crop": random.choice([True, False]),
            "hash": True
        },
        "params": {
            "denoise": {"h": 10, "template_window": 7},
            "contrast": {"alpha": 1.2, "beta": 10},
            "resize": {"width": 1024},
            "normalize": {"clipLimit": 2.0, "tileGrid": (8, 8)},
            "crop": {"margin": 5}
        }
    }

# Example random config
print(generate_random_config())

## 6. Defining the LLM Prompt Template
Template for requesting adapted pipeline parameters based on image metrics.

In [ ]:
# Example of how the prompt is structured for the LLM to decide next steps
prompt_template = '''
An image was processed and the following metrics were evaluated:
- Blur reduction: {blur_reduction:.2f}
- OCR gain: {ocr_gain}
- Overall score: {score:.2f}

Initial parameters:
{params_json}

Decide which steps to enable. Respond only with JSON.'''

print(prompt_template)

## 7. LLM Adjustment Callback
Function to call the LLM and parse its JSON response. Falls back to random config if parsing fails.

In [ ]:
def llm_adjust_callback(data, model_name="llama3.2:11b", custom_prompt=None):
    # Use default prompt if none provided
    prompt = custom_prompt or prompt_template.format(
        blur_reduction=data['metrics']['blur_reduction'],
        ocr_gain=data['metrics']['ocr_gain'],
        score=data['metrics']['score'],
        params_json=json.dumps(data['params'], indent=2)
    )
    response = chat(model=model_name, messages=[{"role": "user", "content": prompt}])
    try:
        return json.loads(response['message']['content'])
    except Exception:
        return generate_random_config()

## 8. Image Quality Evaluation
Compute blur variance and OCR text length to score improvements.

In [ ]:
def evaluate_image_quality(orig, proc):
    b_orig = cv2.Laplacian(cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()
    b_proc = cv2.Laplacian(cv2.cvtColor(proc, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()
    text_orig = len(pytesseract.image_to_string(orig))
    text_proc = len(pytesseract.image_to_string(proc))
    return {
        'blur_reduction': b_proc - b_orig,
        'ocr_gain': text_proc - text_orig,
        'score': (b_proc - b_orig) + (text_proc - text_orig)
    }

## 9. Pipeline Optimization Loop
Iteratively sample configurations and use the LLM feedback to find the best pipeline.

In [ ]:
def optimize_pipeline(orig_image, param_space, llm_call, max_iter=3):
    best = {'score': -float('inf')}
    for _ in range(max_iter):
        # Randomly sample parameters
        current_params = {
            k: {sk: random.choice(v) for sk, v in sp.items()}
            for k, sp in param_space.items()
        }
        # Enable all steps initially
        config = {
            "use": dict.fromkeys(current_params.keys(), True),
            "params": current_params
        }
        steps = build_pipeline(config)
        img, _ = process_image(orig_image, steps)
        metrics = evaluate_image_quality(orig_image, img)

        # Get LLM-adjusted config
        new_config = llm_call({
            "orig": orig_image,
            "proc": img,
            "metrics": metrics,
            "params": current_params
        })
        new_steps = build_pipeline(new_config)
        new_img, history = process_image(orig_image, new_steps)
        new_metrics = evaluate_image_quality(orig_image, new_img)

        # Update best
        if new_metrics['score'] > best['score']:
            best.update({
                'image': new_img,
                'config': new_config,
                'metrics': new_metrics,
                'history': history,
                'score': new_metrics['score']
            })
    return best

## 10. Running Optimization and Visualizing Results
Apply the optimization to an example image and display before/after along with step history.

In [ ]:
# Load example image
orig = cv2.imread('path_to_example_image.jpg')
best_result = optimize_pipeline(orig, param_space, llm_adjust_callback)

print("Best Configuration:\n", json.dumps(best_result['config'], indent=2))
print("Metrics:\n", best_result['metrics'])

# Visualization
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
plt.title("Original")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(best_result['image'], cv2.COLOR_BGR2RGB))
plt.title("Optimized by LLM")
plt.axis("off")
plt.show()

# History of applied steps
for entry in best_result['history']:
    print(entry)